<a href="https://colab.research.google.com/github/Snapchik/chat_analysis/blob/main/Chats_copmparisson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import string
import nltk

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming

In [2]:
# download the stopwords from NLTK
nltk.download('stopwords')

nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
hammas_chat = pd.read_json('/content/drive/My Drive/Colab Notebooks/result.json')

hammas_chat['hashtags'] = [[] for _ in range(len(hammas_chat))]
hammas_chat['date'] = [[] for _ in range(len(hammas_chat))]
hammas_chat['clear_message'] = [[] for _ in range(len(hammas_chat))]

def extract_hashtags(messages):
    try:
        return re.findall(r'#(\w+)', str(messages))[0]
    except IndexError as err:
        return np.nan


hammas_chat['hashtags'] = hammas_chat['messages'].apply(extract_hashtags)

message_count = len(hammas_chat['messages'])

for index in range(message_count):
  hammas_chat['date'][index] = pd.to_datetime(hammas_chat.iloc[index, 3]['date'])

for index in range(message_count):
  hammas_chat['clear_message'][index] = hammas_chat.iloc[index, 3]['text']



hammas_chat = hammas_chat.drop(columns=['name', 'type', 'id'])
hammas_chat = hammas_chat.replace('', np.nan)
hammas_chat = hammas_chat.apply(lambda y: np.nan if len(y)==0 else y)
hammas_chat = hammas_chat.dropna(subset = ['clear_message'])

Streaming output truncated to the last 5000 lines.
<ipython-input-6-05253a1a3b2b>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hammas_chat['clear_message'][index] = hammas_chat.iloc[index, 3]['text']
<ipython-input-6-05253a1a3b2b>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hammas_chat['clear_message'][index] = hammas_chat.iloc[index, 3]['text']
<ipython-input-6-05253a1a3b2b>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

In [7]:
# WHY THIS CODE WORKS AND NOT THE ONE ON TOP
for j in range(len(hammas_chat)):
  lst = []
  for i in hammas_chat.iloc[j, 3]:
    try:
      lst.extend(i.values())
    except AttributeError as err:
      lst.extend(i)
  lst = ''.join(lst)
  #getting rid of typetext/n
  typetext = '\w+\n'
  re.sub(typetext,'', lst)
  hammas_chat.iloc[j, 3] = lst

In [8]:
#Big preprocessing steps

#clearing out some common misspellings and hyperlink
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace('\n', '')
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace('text_link', '')
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace(r'https?://[^\s\n\r]+', '', regex=True)
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace(r'bold\B', '', regex=True)
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace(r'#', '',regex=True)
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace('hashtag', '')
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace('italic', '')
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace('_', '')
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace('—', '')

#applying tokenizer
hammas_chat['messages_token'] = hammas_chat["clear_message"].apply(nltk.word_tokenize)

#removing stop words
stopwords_english = stopwords.words('english')

hammas_chat['tokens_without_stopwords'] = hammas_chat['messages_token'].apply(lambda x:
    [word for word in x if word not in (stopwords_english) and word not in string.punctuation])


# Instantiate stemming class
stemmer = PorterStemmer()

hammas_chat['stemmed_tokens_wt_stopwords_punctuation'] = hammas_chat['tokens_without_stopwords'].apply(lambda x: [stemmer.stem(word) for word in x])

print(hammas_chat['stemmed_tokens_wt_stopwords_punctuation'])

Next up I need to create a util package like in lab